In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

### Config

In [7]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20231113'
pro_file_name = 'ACO Builder MSSP Prospective Explorer v2024.3.txt'
retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2024.3.txt'

### Pro load

In [9]:
pro_df = pd.read_csv(f'{data_dir}/{current_folder}/{pro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
# pro_df = cb_utils.df_format_columns(pro_df)
print(f'{pro_df.shape[0]} rows, {pro_df.shape[1]} columns')
pro_df.head()

614044 rows, 160 columns


,PAC_ID,DeID_TIN,year,program_type,Provider_Name,MSSP_ACO_Name,REACH_ACO_Name,Practice_State,Practice_MSA,Practice_ST_MSA,Practice_MSA_Name,Practice_Zip,Mailing_State,Mailing_MSA,Mailing_ST_MSA,Mailing_MSA_Name,Mailing_Zip,Spec_Count,NP_Count,CNS_Count,PCP_Count,PA_Count,Other_Count,Unique_members,PYs_ESRD,PYs_DIS,PYs_AD,PYs_AND,PYs_Total,RS_ESRD,RS_DIS,RS_AD,RS_AND,Risk_Composite,RS_ESRD_Demog,RS_DIS_Demog,RS_AD_Demog,RS_AND_Demog,Risk_Composite_Demog,Paid_ESRD,Paid_DIS,Paid_AD,Paid_AND,Paid_Composite,Reg_Ret_Exp_ESRD,Reg_Ret_Exp_Dis,Reg_Ret_Exp_AD,Reg_Ret_Exp_AND,Reg_Ret_Exp_Composite,Regional_Efficiency_Ret,Reg_Pro_Exp_ESRD,Reg_Pro_Exp_DIS,Reg_Pro_Exp_AD,Reg_Pro_Exp_AND,Reg_Pro_Exp_Composite,Regional_Efficiency_Pro,Prof_Paid,OP_Paid,IP_Paid,HOME_HEALTH_PAID,SNF_Paid,Hospice_Paid,Anc_Paid,ER_Paid,Sur_Anes_Paid,Rad_Path_Paid,Visits_Paid,Pharm_Paid,Dial_Paid,Other_Prof_Paid,PQEM_Cost,Prof_Paid_Keep,OP_Paid_Keep,Sur_Anes_Paid_Keep,Rad_Path_Paid_Keep,Visits_Paid_Keep,Pharm_Paid_Keep,Dial_Paid_Keep,Other_Prof_Paid_Keep,PQEM_Cost_Keep,OP_Visits,IP_Admits,IP_Days,Home_Health_Visits,Home_Health_Days,SNF_Admits,SNF_Days,Hospice_Visits,Hospice_Days,Anc_Services,ER_Visits,Sur_Anes_Procs,Rad_Path_Procs,Pharm_Procs,Dial_Services,Other_Procs,OP_Visits_Keep,Sur_Anes_Procs_Keep,Rad_Path_Procs_Keep,Pharm_Procs_Keep,Dial_Services_Keep,Other_Procs_Keep,AWV,AWV_Keep,edcnnpa,edcnpa,epct,noner,injury,psych,alcohol,drug,total_ER,unclassified,Readmissions,unique_potl_members,PYs_Potl_ESRD,PYs_Potl_DIS,PYs_Potl_AD,PYs_Potl_AND,paid_potl_esrd,paid_potl_dis,paid_potl_ad,paid_potl_and,rs_potl_esrd,rs_potl_dis,rs_potl_ad,rs_potl_and,PYs_PCP_ESRD,PYs_PCP_DIS,PYs_PCP_AD,PYs_PCP_AND,paid_pcp_esrd,paid_pcp_dis,paid_pcp_ad,paid_pcp_and,rs_pcp_esrd,rs_pcp_dis,rs_pcp_ad,rs_pcp_and,PYs_SPC_ESRD,PYs_SPC_DIS,PYs_SPC_AD,PYs_SPC_AND,paid_spc_esrd,paid_spc_dis,paid_spc_ad,paid_spc_and,rs_spc_esrd,rs_spc_dis,rs_spc_ad,rs_spc_and,PYs_Office,PYs_Outpatient,PYs_SNF,PYs_NF,PYs_HCBS,PYs_ALF,PYs_FQHC_RHC,PYs_Other
0,1052200078,17C41BF897E3,2016,MSSP Prospective,ROGER A LEWIS,NaN,NaN,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1.0,42.166667,20.333333,119.416667,182.916667,0.985946,1.136570,1.186580,1.178582,1.162855,0.553883,0.987882,0.975584,1.063797,1.004289,114239.236530,9905.839805,24785.188946,10240.665844,12348.831908,98387.797438,9053.971949,14992.844146,9115.638033,10228.271878,1.038241,97246.172745,8489.374346,14399.699169,8721.036391,9767.389897,1.087231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,1.0,44.166667,21.333333,127.333333,114239.236530,9670.312879,23818.625577,9879.324735,0.985946,1.115296,1.187055,1.184122,1.0,42.166667,20.333333,119.416667,114239.236530,9905.839805,24785.188946,10240.665844,0.985946,1.136570,1.186580,1.178582,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182.916667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,1052200078,17C41BF897E3,2017,MSSP Prospective,ROGER A LEWIS,NaN,NaN,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1.0,39.500000,21.750000,113.500000,175.750000,1.159624,1.148223,1.049866,1.238855,1.187538,0.556077,1.037245,0.953944,1.071340,1.020802,98094.915918,9144.284893,15363.559353,10806.700216,11493.667480,117700.921930,9713.458354,17210.376617,9459.006867,11020.390710,0.878242,107802.810120,8971.920659,16478.170088,8895.985833,10344.828747,0.935595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197.0,2.0,43.583333,25.083333,119.583333

In [12]:
pro_df.year.value_counts()

2016    97252
2017    93438
2018    90459
2019    87347
2020    84489
2021    82160
2022    78899
Name: year, dtype: int64

In [14]:
for yr in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    pro_df.loc[pro_df.year == yr].to_csv(f'{data_dir}/{current_folder}/pro_{yr}.csv',index=False)

In [11]:
ret_df = pd.read_csv(f'{data_dir}/{current_folder}/{retro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
ret_df = cb_utils.df_format_columns(ret_df)
print(f'{ret_df.shape[0]} rows, {ret_df.shape[1]} columns')
ret_df.head()

594024 rows, 160 columns


,pac_id,deid_tin,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,mailing_state,mailing_msa,mailing_st_msa,mailing_msa_name,mailing_zip,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,unique_members,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd,rs_dis,rs_ad,rs_and,risk_composite,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_composite_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro,prof_paid,op_paid,ip_paid,home_health_paid,snf_paid,hospice_paid,anc_paid,er_paid,sur_anes_paid,rad_path_paid,visits_paid,pharm_paid,dial_paid,other_prof_paid,pqem_cost,prof_paid_keep,op_paid_keep,sur_anes_paid_keep,rad_path_paid_keep,visits_paid_keep,pharm_paid_keep,dial_paid_keep,other_prof_paid_keep,pqem_cost_keep,op_visits,ip_admits,ip_days,home_health_visits,home_health_days,snf_admits,snf_days,hospice_visits,hospice_days,anc_services,er_visits,sur_anes_procs,rad_path_procs,pharm_procs,dial_services,other_procs,op_visits_keep,sur_anes_procs_keep,rad_path_procs_keep,pharm_procs_keep,dial_services_keep,other_procs_keep,awv,awv_keep,edcnnpa,edcnpa,epct,noner,injury,psych,alcohol,drug,total_er,unclassified,readmissions,unique_potl_members,pys_potl_esrd,pys_potl_dis,pys_potl_ad,pys_potl_and,paid_potl_esrd,paid_potl_dis,paid_potl_ad,paid_potl_and,rs_potl_esrd,rs_potl_dis,rs_potl_ad,rs_potl_and,pys_pcp_esrd,pys_pcp_dis,pys_pcp_ad,pys_pcp_and,paid_pcp_esrd,paid_pcp_dis,paid_pcp_ad,paid_pcp_and,rs_pcp_esrd,rs_pcp_dis,rs_pcp_ad,rs_pcp_and,pys_spc_esrd,pys_spc_dis,pys_spc_ad,pys_spc_and,paid_spc_esrd,paid_spc_dis,paid_spc_ad,paid_spc_and,rs_spc_esrd,rs_spc_dis,rs_spc_ad,rs_spc_and,pys_office,pys_outpatient,pys_snf,pys_nf,pys_hcbs,pys_alf,pys_fqhc_rhc,pys_other
0,1052200078,17C41BF897E3,2016,MSSP Retrospective,ROGER A LEWIS,NaN,NaN,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764.0,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1.0,43.250000,21.250000,126.666667,192.166667,0.985946,1.109109,0.917477,1.102973,1.063574,0.553883,1.017221,0.941620,1.075105,1.011056,114239.236530,9131.048942,18090.455625,9427.736381,10864.315145,98387.797438,8987.012896,14591.821944,9083.485434,10184.985109,1.002938,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,228.0,2.916667,52.833333,33.166667,131.916667,79568.107102,8705.224655,24036.309002,9660.629343,0.932560,1.108019,1.233405,1.114614,1.0,43.250000,21.250000,126.666667,114239.236530,9131.048942,18090.455625,9427.736381,0.985946,1.109109,0.917477,1.102973,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,191.166667,0.0,0.0,1.000000,0.0,0.0,0.0,0.0
1,1052200078,17C41BF897E3,2017,MSSP Retrospective,ROGER A LEWIS,NaN,NaN,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764.0,West Virginia,34060.0,WV-34060,"Morgantown, WV",26764,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1.0,39.583333,27.833333,118.416667,186.833333,1.159624,0.865147,1.114838,1.247539,1.179324,0.556077,1.004598,0.953299,1.060740,1.011112,98094.915918,5406.844027,14153.570153,11708.229983,11199.855995,117700.921930,9680.434656,17210.505341,9442.875910,10878.904781,0.872960,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210.0,2.000000,43.583333,30.833333,124.416667,82649.507041,5308.700842,15881.544391,11488.847718,1.091638,0.917781,1.070285,1.244427,1.0,39.583333,27.

In [15]:
ret_df.year.value_counts()

2016    93392
2017    90403
2018    87384
2019    84748
2020    82510
2021    79560
2022    76027
Name: year, dtype: int64

In [ ]:
for yr in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    ret_df.loc[ret_df.year == yr].to_csv(f'{data_dir}/{current_folder}/ret_{yr}.csv',index=False)

# Did not use below

In [6]:
conn = cb_utils.get_engine()
pro_df.to_sql(f'milliman_pro_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # , if_exists='replace'

613556

### Retro load

In [33]:

ret_df.to_csv('ret.csv',index=False)

In [10]:
conn = cb_utils.get_engine()
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # if_exists='replace', 

594838

### Pivots

In [11]:
def pivot_ma_plan_enrollment(df):
    dfs = []
    for i in range(1, 16):
        name, enrollment = f'ma_plan_{i}_name', f'ma_plan_{i}_enrollment'
        plan = df[['deid_tin', 'pac_id', 'year', name, enrollment]]
        plan = plan.loc[~(plan[name].isna()) & ~(plan[enrollment].isna())]
        plan.columns = ['deid_tin', 'pac_id', 'year', 'ma_plan_name', 'ma_enrollment']
        dfs.append(plan)
    return pd.concat(dfs)

In [12]:
ret_pivot_df = pivot_ma_plan_enrollment(ret_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
ret_pivot_df.head()

In [13]:
pro_pivot_df = pivot_ma_plan_enrollment(pro_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
pro_pivot_df.head()

### Summaries 
### Prospective

In [ ]:
s1 = ret_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s1.head()

### Retro

In [ ]:
s2 = pro_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s2.head()

### Prospective

In [ ]:
s3 = pro_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s3.head()

### Retro

In [ ]:
s4 = ret_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s4.head()

### Outputs

In [ ]:
ret_df.loc[:, ret_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_ret_{current_folder}.csv', index=False)

In [ ]:
pro_df.loc[:, pro_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_pro_{current_folder}.csv', index=False)

In [ ]:
pro_pivot_df.to_csv(f'{data_dir}/{current_folder}/pro_pivot.csv', index=False)
ret_pivot_df.to_csv(f'{data_dir}/{current_folder}/ret_pivot.csv', index=False)

In [ ]:
with pd.ExcelWriter(f'{data_dir}/{current_folder}/summary.xlsx') as writer:
    s1.to_excel(writer, sheet_name='retrospective', index=False)
    s2.to_excel(writer, sheet_name='prospective', index=False)
    s3.to_excel(writer, sheet_name='prospective_pivot', index=False)
    s4.to_excel(writer, sheet_name='retrospective_pivot', index=False)